In [5]:
import pandas as pd
import os
year_data_2023 = pd.read_csv("../../Region_Creation_Parsing/BikeDataJanuary.csv")

NameError: name '__file__' is not defined

In [16]:
year_data_2023['start_time'] = pd.to_datetime(year_data_2023['start_time'], errors='coerce')
year_data_2023['end_time'] = pd.to_datetime(year_data_2023['end_time'], errors='coerce')

# --checking how which months are present in the data 
#unique_months = year_data_2023['month'].unique()
#print(sorted(unique_months))

[np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11)]


In [11]:
import numpy as np

# Extract the hour from the datetime columns
year_data_2023['start_hour'] = year_data_2023['start_time'].dt.hour
year_data_2023['end_hour'] = year_data_2023['end_time'].dt.hour
year_data_2023['date'] = year_data_2023['start_time'].dt.date

# Group data to calculate outflow (bikes leaving)
outflow = year_data_2023.groupby(['start_station_id', 'start_station_name', 'date', 'start_hour']).agg(
    bikes_leaving=('trip_id', 'count')
).reset_index()

# Group data to calculate inflow (bikes arriving)
inflow = year_data_2023.groupby(['end_station_id', 'end_station_name', 'date', 'end_hour']).agg(
    bikes_arriving=('trip_id', 'count')
).reset_index()

# Rename columns in inflow for consistency
inflow.rename(columns={
    'end_station_id': 'station_id',
    'end_station_name': 'station_name',
    'end_hour': 'hour'
}, inplace=True)

outflow.rename(columns={
    'start_station_id': 'station_id',
    'start_station_name': 'station_name',
    'start_hour': 'hour'
}, inplace=True)

# Create a DataFrame for all hours, dates, and stations
all_hours = pd.DataFrame({
    'hour': np.tile(np.arange(24), len(year_data_2023['date'].unique())),
    'date': np.repeat(year_data_2023['date'].unique(), 24)
})

# Generate all combinations of stations, dates, and hours
all_combinations = year_data_2023[['start_station_id', 'start_station_name']].drop_duplicates()
all_combinations.rename(columns={
    'start_station_id': 'station_id',
    'start_station_name': 'station_name'
}, inplace=True)

all_combinations = all_combinations.merge(all_hours, how='cross')

# Merge with outflow and inflow data
full_outflow = all_combinations.merge(outflow, on=['station_id', 'station_name', 'date', 'hour'], how='left').fillna(0)
full_inflow_outflow = full_outflow.merge(inflow, on=['station_id', 'station_name', 'date', 'hour'], how='left').fillna(0)

# Show the result
full_inflow_outflow.head(24)

,station_id,station_name,hour,date,bikes_leaving,bikes_arriving
0,7150,Dufferin St / Sylvan Av (Dufferin Grove Park),0,2023-09-01,1.0,0.0
1,7150,Dufferin St / Sylvan Av (Dufferin Grove Park),1,2023-09-01,0.0,0.0
2,7150,Dufferin St / Sylvan Av (Dufferin Grove Park),2,2023-09-01,0.0,0.0
3,7150,Dufferin St / Sylvan Av (Dufferin Grove Park),3,2023-09-01,0.0,2.0
4,7150,Dufferin St / Sylvan Av (Dufferin Grove Park),4,2023-09-01,0.0,0.0
5,7150,Dufferin St / Sylvan Av (Dufferin Grove Park),5,2023-09-01,0.0,0.0
6,7150,Dufferin St / Sylvan Av (Dufferin Grove Park),6,2023-09-01,0.0,0.0
7,7150,Dufferin St / Sylvan Av (Dufferin Grove Park),7,2023-09-01,1.0,0.0
8,7150,Dufferin St / Sylvan Av (Dufferin Grove Park),8,2023-09-01,1.0,0.0
9,7150,Dufferin St / Sylvan Av (Dufferin Grove Park),9,2023-09-01,1.0,1.0


In [14]:
#full_inflow_outflow.to_csv('/Users/weinbergershane/Local Documents/WQC/Exported Data/inflow-outflow.csv', index=False)

In [1]:
# Calculate total inflow (bikes_arriving) and outflow (bikes_leaving) by station for the filtered data
station_summary = full_inflow_outflow.groupby(['station_id', 'station_name']).agg(
    total_leaving=('bikes_leaving', 'sum'),
    total_arriving=('bikes_arriving', 'sum')
).reset_index()

# Sort by total bikes leaving or arriving (you can adjust this based on what you're interested in)
station_summary_sorted = station_summary.sort_values(by='total_leaving', ascending=False)

station_summary_sorted.head()

NameError: name 'full_inflow_outflow' is not defined